
## <p style="text-align: center;">MIS 284N - Big Data and Distributed Programming</p>
## <p style="text-align: center;">Roshni Das and Barrett Poth </p>
## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and AWS</p>
## <p style="text-align: center;">Total points: 100</p>
## <p style="text-align: center;">Due: Thursday, October 19 submitted via Canvas by 11:59 pm</p>

Your project should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTEID for both students.

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

This project is on Tensorflow, its usage, Cloud services and why they are helpful in analyzing Big Data.

In this Project, we are trying to analyze crime data. This data is taken from <a href=https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2/data> here </a>. You have to download data from here, it should be around 1.6 GB in size. 

You have to predict type of crime that can happen given the features.

You have to complete all the four tasks and report your observations for each if any. There is a bonus task as well, it has two options and you can choose any one of it.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

# Task 1 - Read the data and filter the data you need

This is open ended, you can use any features you want and drop any one of them. The 'Primary Type' is our target variable which we will be predicting for.
Split the data into training and testing in the ratio of 80:20.

In [2]:
print('Start: Data Read',datetime.datetime.now())
df = pd.read_csv('./Crimes_-_2001_to_present.csv')
print('End: Data Read',datetime.datetime.now())

Start: Data Read 2018-10-22 16:44:29.253686
End: Data Read 2018-10-22 16:45:05.023836


#### Time taken to read data = 65 sec

In [3]:
df.shape

(6721685, 22)

In [4]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [5]:
df=df.dropna(subset=['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'], how='any')

In [6]:
df.shape

(6051694, 22)

In [7]:
df['Primary Type'].unique()

array(['BATTERY', 'OTHER OFFENSE', 'ROBBERY', 'NARCOTICS',
       'CRIMINAL DAMAGE', 'WEAPONS VIOLATION', 'THEFT', 'BURGLARY',
       'MOTOR VEHICLE THEFT', 'PUBLIC PEACE VIOLATION', 'ASSAULT',
       'CRIMINAL TRESPASS', 'CRIM SEXUAL ASSAULT',
       'INTERFERENCE WITH PUBLIC OFFICER', 'ARSON', 'DECEPTIVE PRACTICE',
       'LIQUOR LAW VIOLATION', 'KIDNAPPING', 'SEX OFFENSE',
       'OFFENSE INVOLVING CHILDREN', 'PROSTITUTION', 'HOMICIDE',
       'GAMBLING', 'INTIMIDATION', 'STALKING', 'OBSCENITY',
       'PUBLIC INDECENCY', 'HUMAN TRAFFICKING',
       'CONCEALED CARRY LICENSE VIOLATION', 'OTHER NARCOTIC VIOLATION',
       'NON - CRIMINAL', 'NON-CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'RITUALISM'], dtype=object)

_Okay so I'm assuming that 'Primary Type' is y._

In [8]:
tmp_df = df
y = tmp_df['Primary Type']
X = tmp_df.drop(columns=['Primary Type', 'ID', 'Case Number'])

In [9]:
X=X.drop(columns=['Updated On','Block', 'Community Area', 'Ward', 'X Coordinate', 'Y Coordinate', 'Location','Year'])

# Task 2 - Convert the features in a form that can be given as input to tensorflow library/functions

Here you have to search a little bit. The data that we are using has data other than numerical values. Tensors take two kinds of values, either numeric or categorical. So, you have to convert the given data into a logical form which can be given to tensors.

In [10]:
#Factorizing y_train for each category in y_train and converting into numpy
y_label,y_np=pd.factorize(y)
y_label

array([ 0,  1,  0, ..., 21, 21, 21])

In [11]:
from sklearn import preprocessing
import warnings
le = preprocessing.LabelEncoder() 
lb = preprocessing.LabelBinarizer()

le.fit(y_np.values)
y_encoded = le.transform(y)

lb.fit(y_encoded)
Y_flat = lb.transform(y_encoded)

In [12]:
Y_flat

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Changed Arrest and Domestic columns to 0 and 1 in place of False and True

In [13]:
X['Arrest'] = X['Arrest'] .map({False: 0, True: 1})
X['Domestic'] = X['Domestic'] .map({False: 0, True: 1})
X

,Date,IUCR,Description,Location Description,Arrest,Domestic,Beat,District,FBI Code,Latitude,Longitude
0,03/18/2015 07:44:00 PM,041A,AGGRAVATED: HANDGUN,STREET,0,0,1111,11.0,04B,41.891399,-87.744385
1,03/18/2015 11:00:00 PM,4625,PAROLE VIOLATION,STREET,1,0,725,7.0,26,41.773372,-87.665319
2,03/18/2015 10:45:00 PM,0486,DOMESTIC BATTERY SIMPLE,APARTMENT,0,1,222,2.0,08B,41.813861,-87.596643
3,03/18/2015 10:30:00 PM,0460,SIMPLE,APARTMENT,0,0,225,2.0,08B,41.800802,-87.622619
4,03/18/2015 09:00:00 PM,031A,ARMED: HANDGUN,SIDEWALK,0,0,1113,11.0,03,41.878065,-87.743354
5,03/18/2015 10:00:00 PM,0460,SIMPLE,APARTMENT,0,0,223,2.0,08B,41.805443,-87.604284
6,03/18/2015 11:00:00 PM,0486,DOMESTIC BATTERY SIMPLE,APARTMENT,0,1,733,7.0,08B,41.766403,-87.649296
7,03/18/2015 09:35:00 PM,0486,DOMESTIC BATTERY SIMPLE,APARTMENT,0,1,213,2.0,08B,41.817553,-87.619819
8,03/18/2015 10:09:00 PM,1811,POSS: CANNABIS 30GMS OR LESS,STREET,1,0,912,9.0,18,41.828138,-87.672782
9,03/18/2015 09:25:00 PM,0460,SIMPLE,RESIDENCE PORCH/HALLWAY,0,0,511,5.0,08B,41.717455,-87.617663


Alright let's try to one-hot encode this thing

In [14]:
print('Start: Create Dummies',datetime.datetime.now())
X_dummies = pd.get_dummies(X,
        columns=['IUCR', 'Description', 'Location Description', 'Beat', 'District', 'FBI Code'])
print('End: Create Dummies',datetime.datetime.now())

Start: Create Dummies 2018-10-22 16:45:24.395331
End: Create Dummies 2018-10-22 16:47:12.796477


#### Time taken for one-hot encoding = 108 sec

In [15]:
X_dummies.head()

,Date,Arrest,Domestic,Latitude,Longitude,IUCR_0110,IUCR_0141,IUCR_0142,IUCR_0261,IUCR_0262,...,FBI Code_14,FBI Code_15,FBI Code_16,FBI Code_17,FBI Code_18,FBI Code_19,FBI Code_20,FBI Code_22,FBI Code_24,FBI Code_26
0,03/18/2015 07:44:00 PM,0,0,41.891399,-87.744385,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,03/18/2015 11:00:00 PM,1,0,41.773372,-87.665319,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,03/18/2015 10:45:00 PM,0,1,41.813861,-87.596643,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,03/18/2015 10:30:00 PM,0,0,41.800802,-87.622619,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,03/18/2015 09:00:00 PM,0,0,41.878065,-87.743354,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Lets look at time features

In [16]:
from datetime import datetime

In [17]:
X_dummies.Date[0]

'03/18/2015 07:44:00 PM'

In [18]:
datetimes = [datetime.strptime(d, '%m/%d/%Y %I:%M:%S %p') for d in X_dummies.Date]

In [19]:
#Changing Date column into a number and replacing the column with these values in the dataset
monthyear=[]
for i in range(len(datetimes)):
    concat=(datetimes[i].year*100 +(datetimes[i].month))
    monthyear.append(concat)
X_dummies.Date = monthyear

In [20]:
import datetime
print('Start: Split Data',datetime.datetime.now())
X_train, X_test, y_train, y_test = train_test_split(X_dummies, Y_flat, test_size=0.20, random_state=42)
print('End: Split Data',datetime.datetime.now())

Start: Split Data 2018-10-22 16:48:37.742044
End: Split Data 2018-10-22 16:51:09.732251


#### Time taken to split data to train and test= 92 sec

In [21]:
X_train.shape

(4841355, 1313)

In [22]:
y_train.shape

(4841355, 34)

# Task 3 - Build the model

Once the features and tensors are made, you have to build a machine learning model which, given the features predicts the type of crime. 
Report the accuracy of the model.

### Model ###

Taking 1 million rows into consideration to train the model since kernel is dying when model is supplied with 4 million rows of data

In [23]:
X_train=X_train[:1000000]

In [24]:
y_train=y_train[:1000000]

In [25]:
X_train.shape

(1000000, 1313)

In [26]:
y_train.shape

(1000000, 34)

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xlrd
from tensorflow.python.framework import ops
ops.reset_default_graph()

#Create weight and bias
w = tf . Variable (tf.random_normal(shape=[1313,34]), name = "w") 
b = tf . Variable (tf.random_normal(shape=[34]), name = "b") 

#Create placeholders
n_samples =X.shape[0]
x_data=tf.placeholder(tf.float32,[None,1313],name="X")
y_target =tf.placeholder(tf.float32,[None,34],name="Y")

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#Define model and loss
mod = tf.matmul(x_data, w) + b
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=mod, labels=y_target))

# Define the learning rate， batch_size etc.
batch_size = 30
iter_num = 10

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize (loss) 

# Define the accuracy
# The default threshold is 0.5, rounded off directly
prediction = tf.round(tf.sigmoid(mod))
# Bool into float32 type
correct = tf.cast(tf.equal(prediction, y_target), dtype=tf.float32)
# Average
accuracy = tf.reduce_mean(correct)
# End of the definition of the model framework


# Start training model
# Define the variable that stores the result
loss_trace = []
train_acc = []
test_acc = []



In [28]:
print('Start: Train and Test Model',datetime.datetime.now())
# Training model
for epoch in range(iter_num):
    # Generate random batch index
   # batch_index = np.random.choice(len(X_train), size=batch_size)
    batch_train_X = X_train
    batch_train_y = np.matrix(y_train)
    sess.run(optimizer, feed_dict={x_data: batch_train_X, y_target: batch_train_y})
    temp_loss = sess.run(loss, feed_dict={x_data: batch_train_X, y_target: batch_train_y})
    print('convert into a matrix, and the shape of the placeholder to correspond')
    temp_train_acc = sess.run(accuracy, feed_dict={x_data: X_train, y_target: np.matrix(y_train)})
    temp_test_acc = sess.run(accuracy, feed_dict={x_data: X_test, y_target: np.matrix(y_test)})
    # recode the result
    loss_trace.append(temp_loss)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    # output
    #if (epoch + 1) % 30 == 0:
    print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(epoch + 1, temp_loss,
                                                                          temp_train_acc, temp_test_acc))

print('End: Train and Test Model',datetime.datetime.now())

Start: Train and Test Model 2018-10-22 16:51:15.659615
convert into a matrix, and the shape of the placeholder to correspond
epoch:    1 loss: 277822.000000 train_acc: 0.739556 test_acc: 0.739603
convert into a matrix, and the shape of the placeholder to correspond
epoch:    2 loss: 263119.218750 train_acc: 0.941802 test_acc: 0.941800
convert into a matrix, and the shape of the placeholder to correspond
epoch:    3 loss: 221601.328125 train_acc: 0.941385 test_acc: 0.941381
convert into a matrix, and the shape of the placeholder to correspond
epoch:    4 loss: 181168.234375 train_acc: 0.970588 test_acc: 0.970588
convert into a matrix, and the shape of the placeholder to correspond
epoch:    5 loss: 163210.125000 train_acc: 0.953574 test_acc: 0.953521
convert into a matrix, and the shape of the placeholder to correspond
epoch:    6 loss: 204307.296875 train_acc: 0.951912 test_acc: 0.951923
convert into a matrix, and the shape of the placeholder to correspond
epoch:    7 loss: 193571.9843

### Highest accuracy observed : 97.05 %

# Task 4 - Run the model on your laptop and AWS cloud and observe the time taken for reading the data, training and testing. 

Report your observations. Due to more RAM needed for the same, it might be the case that it doesn't run on your laptop, so in that case report your laptop configuration and state it didnt run.

#### Time taken to read data on laptop : 65 sec
#### Time taken to read data on AWS    : 40 sec

#### Time taken on laptop to run the model on training and testing data  :  49 min
#### Time taken on AWS    to run the model on training and testing data  :  37 min

# Bonus Tasks

# Option 1: Try to build a Neural Network model, train on the same features and report the accuracy.
Report your observations.

In [32]:
def multilayer_neuralnetwork(x, weights, biases):#, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    #layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [33]:
n_hidden_1 = 38
n_input = 1313
n_classes = 34

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")

training_epochs =10
display_step = 1
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [35]:
predictions = multilayer_neuralnetwork(x, weights, biases)#, keep_prob)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [37]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_loss = 0.0
        total_batch = int(len(X_train) / batch_size)
        x_batches = np.array_split(X_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, loss], feed_dict={x: batch_x, y: batch_y})#, keep_prob: 0.8})
            avg_loss += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "loss=","{:.9f}".format(avg_loss))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: y_test}))#, keep_prob: 1.0}))

Epoch: 0001 loss= 3292.620134453
Epoch: 0002 loss= 2.410240363
Epoch: 0003 loss= 2.410253280
Epoch: 0004 loss= 2.410254038
Epoch: 0005 loss= 2.410254277
Epoch: 0006 loss= 2.410254385
Epoch: 0007 loss= 2.410254436
Epoch: 0008 loss= 2.410254461
Epoch: 0009 loss= 2.410254474
Epoch: 0010 loss= 2.410254481
Optimization Finished!
Accuracy: 0.20986435


#### Accuracy Observed : 20.98 %

# Option 2: Try to predict other things like Arrest or Description of Crime. 

You can change the features in this task. Report your observations.